**Fire.ca.gov Data Extraction + Preprocessing**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL and years to scrape
base_url = "https://www.fire.ca.gov/incidents/"
years = list(range(2016, 2026))  # 2016 to 2025

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "https://www.fire.ca.gov/"
}

# Storage for all incident data
names = []
locations = []
dates = []
acres = []
containment = []

for year in years:
    url = f"{base_url}{year}"
    response = requests.get(url, headers=header)
    print(response.status_code)
    if response.status_code != 200:
        continue

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", attrs={"class": "table datatable"})
    rows = table.find_all("tr")[1:]

    for row in rows:
      names.append(row.find('a').text.strip())
      locations.append(row.find_all('td')[0].text.strip())
      dates.append(row.find_all('td')[1].text.strip())
      acres.append(row.find_all('td')[2].text.strip().replace(',', ''))
      containment.append(row.find_all('td')[3].text.strip().replace('%', ''))

    time.sleep(1)

200
200
200
200
200
200
200
200
200
200


In [ ]:
df_fire = pd.DataFrame({
    "Name": names,
    "County List": locations,
    "Date": dates,
    "Acres Burned": acres,
    "Containment (%)": containment
})

df_fire = df_fire[df_fire["Acres Burned"] != "External Incident Link"]
df_fire = df_fire[df_fire["Acres Burned"] != ""]
df_fire

,Name,County List,Date,Acres Burned,Containment (%)
0,Soberanes Fire,Monterey,7/22/2016,132127,100
1,Erskine Fire,Kern,6/23/2016,48019,100
2,Chimney Fire,San Luis Obispo,8/13/2016,46235,100
3,Blue Cut Fire,San Bernardino,8/16/2016,36274,100
4,Gap Fire,Siskiyou,8/27/2016,33867,100
...,...,...,...,...,...
2666,Center Fire,San Diego,1/22/2025,7,100
2667,Friars Fire,San Diego,1/21/2025,3,100
2668,Round Fire,Ventura,5/31/2025,2,0
2669,Gilman Fire,San Diego,1/23/2025,2,100


In [ ]:
# Start from the original DataFrame
df_expanded = df_fire.copy()

# Convert Date to datetime and extract Year
df_expanded["Date"] = pd.to_datetime(df_expanded["Date"], errors='coerce')
df_expanded["Year"] = df_expanded["Date"].dt.year

# Split counties into lists
df_expanded["County List"] = df_expanded["County List"].str.split(",\s*")  # assume original col was "County" or rename it as needed

# Count how many counties
df_expanded["County Count"] = df_expanded["County List"].apply(len)

# Divide acres evenly
df_expanded["Acres per County"] = df_expanded["Acres Burned"].astype(float) / df_expanded["County Count"]

# Explode
df_expanded = df_expanded.explode("County List").reset_index(drop=True)

# Rename and clean
df_expanded = df_expanded.rename(columns={
    "County List": "County",
    "Acres per County": "Acres Burned (Per County)"
})

# Drop out-of-state or blank counties
df_expanded = df_expanded[
    (~df_expanded["County"].isin(["State of Oregon", "State of Nevada", ""])) &
    (df_expanded["County"].notna())
]

# Reorder columns to include Year
df_expanded = df_expanded[["Name", "Year", "County", "Acres Burned (Per County)"]]

In [ ]:
df_expanded

,Name,Year,County,Acres Burned (Per County)
0,Soberanes Fire,2016,Monterey,132127.0
1,Erskine Fire,2016,Kern,48019.0
2,Chimney Fire,2016,San Luis Obispo,46235.0
3,Blue Cut Fire,2016,San Bernardino,36274.0
4,Gap Fire,2016,Siskiyou,33867.0
...,...,...,...,...
2660,Center Fire,2025,San Diego,7.0
2661,Friars Fire,2025,San Diego,3.0
2662,Round Fire,2025,Ventura,2.0
2663,Gilman Fire,2025,San Diego,2.0


**Some Fires Go through multiple counties. Therefore, we have Acres Burned Per County**

In [ ]:
df_expanded[df_expanded["Name"] == "Creek Fire"]

,Name,Year,County,Acres Burned (Per County)
60,Creek Fire,2016,Yolo,277.0
118,Creek Fire,2016,Napa,65.0
204,Creek Fire,2017,Los Angeles,15619.0
242,Creek Fire,2017,Stanislaus,1749.0
301,Creek Fire,2017,Fresno,357.0
495,Creek Fire,2017,Kern,33.0
657,Creek Fire,2018,Shasta,1678.0
840,Creek Fire,2018,San Bernardino,33.0
845,Creek Fire,2018,Lake,32.0
849,Creek Fire,2018,Alameda,30.0


**So far, we know the names of Fires, the year it occurred, the counties devastated, and the acres burned (per county)**

**Identify the counties with most risk considering Acres Burned Per County**

In [ ]:
temp_df = df_expanded.groupby("County")["Acres Burned (Per County)"].sum().sort_values(ascending=False)[:10]
temp_df

,Acres Burned (Per County)
County,
Siskiyou,768926.000000
Trinity,712561.976190
Tehama,668503.776190
Butte,599968.300000
Shasta,490897.633333
Plumas,394160.800000
Lassen,376013.800000
Lake,364452.559524
Fresno,322455.500000


In [ ]:
risk_counties = temp_df.index.tolist()
risk_counties

['Siskiyou',
 'Trinity',
 'Tehama',
 'Butte',
 'Shasta',
 'Plumas',
 'Lassen',
 'Lake',
 'Fresno',
 'Mendocino']

**Extract Insurance Data to see if there is a correlation with Acres Burned (Fire Incidence) and insurance premiums in these areas**

**Residential Insurance Policy Analysis by County**

1) Used tabula-py to read all the tables from the pdfs and wrote out a single csv. \\
2) Manually Post-processed table, deleting unnecessary rows and columns that were contained \\

In [ ]:
title = "/content/postprocessed_Insurance_Policy.csv"
df_insurance = pd.read_csv(title)
df_insurance.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/postprocessed_Insurance_Policy.csv'

**This is Insurance Policy Data from 2015-2019 that will get merged**

In [ ]:
filename_2 = "/content/Postprocessed_2015_2019_Insurance_policy_data.csv"
df_2015_2019 = pd.read_csv(filename_2)
df_2015_2019.head()

In [ ]:
df_2015_2019 = df_2015_2019.rename(columns={
    "Fair Plan: Number of New Policies":      "FAIR Plan: Number of New Policies",
    "Fair Plan: Number of Renewed Policies":  "FAIR Plan: Number of Renewed Policies"
})

# 2. Concatenate them
df_all = pd.concat([df_insurance, df_2015_2019], ignore_index=True)

# 3. Ensure Year is numeric and sort for each County descending
df_all["Year"] = df_all["Year"].astype(int)
df_all = (
    df_all
      .sort_values(["County", "Year"], ascending=[True, False])
      .reset_index(drop=True)
)

In [ ]:
df_2015_2019.shape, df_insurance.shape

In [ ]:
df_all

**So Far, this is our insurance data, blocked by county and year, tracking the Voluntary Market policy, FAIR plan Policy, and Difference-in-condition Policy. Important Note is that years 2015-2019 have 0 difference-in-conditions policy (likely started only from 2020 onwards)**

In [ ]:
import pandas as pd

# assume these are your two DataFrames
# df_all: 531 rows × 9 cols (County, Year, Voluntary…, FAIR…, etc.)
# df_expanded:   2639 rows × 4 cols (Name, Year, County, Acres Burned)

# 1. Normalize County names (optional, but recommended)
df_all["County"] = df_all["County"].str.title().str.strip()
df_expanded["County"] = df_expanded["County"].str.title().str.strip()

In [ ]:
# 2. Aggregate fires to one row per County×Year
df_expanded = (
    df_expanded
      .groupby(["County","Year"], as_index=False)
      ["Acres Burned (Per County)"]
      .sum()
      .rename(columns={"Acres Burned (Per County)": "Total Acres Burned"})
)


In [ ]:
df_expanded.head()

In [ ]:
# 3. Merge on County & Year
merged_fires_insurance = (
    df_all
      .merge(df_expanded,
             on=["County","Year"],
             how="left")
      .fillna({"Total Acres Burned": 0})
      .sort_values(["County","Year"], ascending=[True, False])
      .reset_index(drop=True)
)

In [ ]:
merged_fires_insurance.head()

In [ ]:
merged_fires_insurance.to_csv("aggregated_fires_insurance_dataset.csv", header=True)